## Importamos librerías

In [1]:
import os
import pandas as pd
import json
import pickle
import zipfile
import shutil

# Instalamos los paquetes necesarios para que funcione desde la Colab
!pip install avro-python3
!pip install 
!pip install tf_slim==1.1.0
!pip install tf-models-official==2.7.0
!pip install lvis
!pip install tensorflow_io==0.23.1
!pip install keras==2.7.0
!pip install opencv-python-headless==4.5.2.52


  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44010 sha256=5f4bba67164c96084d0cda39213cb3d6b0e591d1db8a0c6961ea1c826f6536c4
  Stored in directory: /root/.cache/pip/wheels/d6/e5/b1/6b151d9b535ee50aaa6ab27d145a0104b6df02e5636f0376da
Successfully built avro-python3
ERROR: You must give at least one requirement to install (see "pip help install")
     |████████████████████████████████| 352 kB 5.0 MB/s 
     |████████████████████████████████| 1.8 MB 5.1 MB/s 
     |████████████████████████████████| 596 kB 63.0 MB/s 
     |████████████████████████████████| 234 kB 72.9 MB/s 
     |████████████████████████████████| 1.1 MB 56.3 MB/s 
     |████████████████████████████████| 47.7 MB 1.3 MB/s 
     |████████████████████████████████| 1.2 MB 42.9 MB/s 
     |████████████████████████████████| 4.9 MB 58.4 MB/s 
     |████████████████████████████████| 99 kB 9.6 MB/s 
     |████████████████████████████████| 90 kB 12.4 MB/s 
     |████████████████████████████████| 4

## Creamos nuestro archivo Label Map

In [2]:
labels = [{'name':'carro', 'id': 1}, {'name': 'motos', 'id': 2}]
with open("label_map.pbtxt", "w") as f:
  for label in labels:
    f.write('item { \n')
    f.write('\tname:\'{}\'\n'.format(label['name']))
    f.write('\tid:{}\n'.format(label['id']))
    f.write('}\n')

## Instalamos la librería de Object Detection en Google Colab

In [3]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
#!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
/content/models
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 http://ppa.launchpad.n

In [4]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

## Clonamos el modelo SSD Mobilenet v2
Si deseas puedes clonar cualquiera de los modelos pre entrenados

In [5]:
# Descargamos los modelos pre-enterenados en este caso SSD + MobileNetV2
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz \
    -O /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2022-02-15 04:05:53--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 173.194.212.128, 2607:f8b0:400c:c11::80
Connecting to download.tensorflow.org (download.tensorflow.org)|173.194.212.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

/content/ssd_mobile 100%[===================>]  19.56M  --.-KB/s    in 0.09s   

2022-02-15 04:05:53 (216 MB/s) - ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



#### Descromprimimos el modelo pre-entrenado

In [6]:
!tar -zxvf /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
output_path = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
output_path = os.path.join(os.getcwd(), output_path)
print("La carpeta se almaceno en {}".format(output_path))

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index
La carpeta se almaceno en /content/models/research/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8


#### Definimos la ruta de nuestro modelo

In [7]:
path_training = '/content/ssd_mobilenet'
os.mkdir(path_training)

## Definimos la ruta del archivo original pipeline.config y en dónde vamos a almacenar nuestro pipeline.config

In [8]:
source_config = "{}/pipeline.config".format(output_path)
target_config = "{}/pipeline.config".format(path_training)
shutil.copyfile(source_config, target_config)

'/content/ssd_mobilenet/pipeline.config'

## Configuramos el archivo pipeline.config
Inicialmente importamos librerías

In [9]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [10]:
# Obtenemos la configuración del archivo pipeline
config = config_util.get_configs_from_pipeline_file(target_config)


In [11]:
# Visualizamos la configuración base
config

{'eval_config': metrics_set: "coco_detection_metrics"
 use_moving_averages: false,
 'eval_input_config': label_map_path: "PATH_TO_BE_CONFIGURED"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED"
 },
 'eval_input_configs': [label_map_path: "PATH_TO_BE_CONFIGURED"
 shuffle: false
 num_epochs: 1
 tf_record_input_reader {
   input_path: "PATH_TO_BE_CONFIGURED"
 }
 ],
 'model': ssd {
   num_classes: 90
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.9969999790191

In [12]:
# Creamos una variable proto_str para poder modificar las variables del archivo pbtxt
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(target_config, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

In [13]:
pipeline_config

model {
  ssd {
    num_classes: 90
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0


In [14]:
# Definimos las rutas en donde se encuentra los TFRecords y el label map para agregarlos al archivo de configuración del pipeline.config
label_map_pbtxt_fname = "/content/label_map.pbtxt"
train_record_fname = "/content/train.record"
test_record_fname = "/content/test.record"

In [15]:
# Cantidad de clases del modelo
pipeline_config.model.ssd.num_classes = 2

# El tamaño del batch size, entre más grande mas costo computacional va a necesitar en el proceso de entrenamiento, pero a su vez entrenara más rapido.
pipeline_config.train_config.batch_size = 4

# Donde almacenaremos los resultados del entrenamiento
pipeline_config.train_config.fine_tune_checkpoint ="{}/checkpoint/ckpt-0".format(output_path)

# Qué tipo de deteción aplicaremos (Object detection)
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

# Dirección del label map
pipeline_config.train_input_reader.label_map_path = label_map_pbtxt_fname

# Dirección del train TFRecord
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_fname

# Dirección del label map
pipeline_config.eval_input_reader[0].label_map_path = label_map_pbtxt_fname

# Dirección del test TFRecord
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_fname

In [16]:
# Visualizamos nuestro pipeline_config final.
pipeline_config

model {
  ssd {
    num_classes: 2
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
 

In [17]:
# Almacenamos nuestro archivo final
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(target_config, "wb") as f:
  f.write(config_text)

## Entrenamiento de modelo de object detection

In [ ]:
num_steps = 5000
model_dir = "/content/ssd_mobilenet"

# Utilizamos directamente la libreria de object detection para ejecutar el script model_main_tf2
# Los argumentos principales son el archivo de configuración, la ubicación del modelo y la cantidad de steps que ejecutará.
!python /content/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path={target_config} \
--model_dir={model_dir} \
--num_train_steps={num_steps}

2022-02-15 04:06:02.654333: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0215 04:06:02.666415 140398825326464 mirrored_strategy.py:376] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 5000
I0215 04:06:02.673085 140398825326464 config_util.py:552] Maybe overwriting train_steps: 5000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0215 04:06:02.673251 140398825326464 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0215 04:06:02.729390 140398825326464 deprecation.py:347] From /content/models/research/object_detection/model_lib_v2.py:564: StrategyBase.experimental_distribute_

## Análisis de los resultados con TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/ssd_mobilenet"

## Exportación de modelo

Exportamos nuestro modelo para poder utilizarlo en cualquier otro momento.

In [ ]:

output_directory = "/content/fine_tuned_model"

!python /content/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path {target_config} \
--trained_checkpoint_dir {model_dir} \
--output_directory {output_directory}

## Compresión del modelo

Comprimimos el modelo y lo descargamos para tenerlo local.

In [ ]:
!zip -r /content/fine_tuned_model.zip /content/fine_tuned_model